In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types
import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
        .appName('test') \
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/21 12:44:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('data/raw/green/*/*')
df_green.registerTempTable('green')

In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 01:00:00|  24|126.41999999999999|             6|
|2020-01-01 05:00:00| 244|             25.26|             2|
|2020-01-01 07:00:00| 112|             66.76|             3|
|2020-01-01 10:00:00| 169|              6.12|             1|
|2020-01-01 11:00:00| 193|              14.3|             2|
|2020-01-01 15:00:00|  25|            192.12|            11|
|2020-01-01 19:00:00| 196|36.400000000000006|             3|
|2020-01-02 14:00:00| 228|             96.46|             4|
|2020-01-02 20:00:00| 146|             15.66|             2|
|2020-01-02 23:00:00|  52|              21.3|             1|
|2020-01-03 03:00:00| 179|               7.3|             1|
|2020-01-03 08:00:00| 212|              16.8|             1|
|2020-01-03 10:00:00|  29|             163.8|             7|
|2020-01-03 11:00:00| 12

In [8]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [9]:
df_yellow = spark.read.parquet('data/raw/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 01:00:00|  24| 783.8999999999999|            44|
|2020-01-01 04:00:00| 202|              42.1|             2|
|2020-01-01 05:00:00| 244|185.17000000000004|            17|
|2020-01-01 05:00:00|  71|             49.71|             1|
|2020-01-01 07:00:00|  48|           2857.21|           120|
|2020-01-01 07:00:00| 179|            119.05|             7|
|2020-01-01 09:00:00|  93|287.46000000000004|             4|
|2020-01-01 10:00:00| 169|              19.1|             2|
|2020-01-01 11:00:00| 193|            105.66|             4|
|2020-01-01 15:00:00|  25|             14.52|             1|
|2020-01-01 19:00:00| 196|             152.8|             2|
|2020-01-01 21:00:00| 157|             61.42|             1|
|2020-01-02 03:00:00| 143| 82.77999999999999|             4|
|2020-01-02 04:00:00| 26

In [12]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [13]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [14]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [15]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [16]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  24|  

In [17]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')